In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_rate = pd.read_csv('/kaggle/input/anime-recommendations-database/rating.csv')
df_anime = pd.read_csv('/kaggle/input/anime-recommendations-database/anime.csv')

In [ ]:
Name = list(df_anime["type"].unique())
Qnt = [len(df_anime[df_anime["type"] == e]) for e in Name]
Name = Name[:-1]
Qnt = Qnt[:-1]

In [ ]:
print(Name)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_title('Type of anime')
ax.set_ylabel('Number of anime')
ax.bar(Name,Qnt)

for index,data in enumerate(Qnt):
    plt.text(x=index-0.25 , y =data+30, s=f"{data}" , fontdict=dict(fontsize=12))
plt.show()

In [ ]:
Genre = dict()

for i in range(len(df_anime)):
    str_genre = df_anime.loc[i, "genre"]
    if pd.isnull(str_genre):
        print(i,end=" ")
        continue
    Arr = str_genre.strip().split(", ")
    for j in Arr:
        if j not in Genre:
            Genre[ j ] = 1
        else :
            Genre[ j ] += 1
            
_name = sorted(list(Genre.keys()) ,key = lambda x:-Genre[x])
_value = [ Genre[e] for e in _name]


In [ ]:
_name[:10],_value[:10]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,5))
ax = fig.add_axes([0,0,1,1])
ax.set_title('Top 10 Genre of anime')
ax.barh(_name[:10][::-1],_value[:10][::-1])
ax.set_xlabel('Number of anime')
plt.show()

## Data Processing

In [ ]:
 df_anime[df_anime["type"] == "Music"]

In [ ]:
df_anime[df_anime["type"] == 'OVA']["anime_id"].values

In [ ]:
set_tv = set(df_anime[df_anime["type"] == 'TV']["anime_id"].values)
set_movie = set(df_anime[df_anime["type"] == 'Movie']["anime_id"].values)
set_ova = set(df_anime[df_anime["type"] == 'OVA']["anime_id"].values)

In [ ]:
dType = {'Movie':[], 'TV':[], 'OVA':[], 'Special':[], 'Music':[], 'ONA':[]}
grouped = df_rate.groupby("user_id")
k = 0
who_skipped = [46809]
#limit_rate = 100000

for i in range(73515):
    if i+1 in who_skipped:
        continue
    A = grouped.get_group(i+1)
    
    #for j in range(len(A)):
    B = A[A['rating']>=1]
    set_trans = set(B['anime_id'].values)
    
    adding_tv = list(set_tv.intersection(set_trans)) 
    if len(adding_tv)>1:
        dType['TV'].append(adding_tv)

    adding_movie = list(set_movie.intersection(set_trans)) 
    if len(adding_movie)>1:
        dType['Movie'].append(adding_movie)

    adding_ova = list(set_ova.intersection(set_trans)) 
    if len(adding_ova)>1:
        dType['OVA'].append(adding_ova)

        #print(k)
    #records.append(grouped.get_group(i+1)['anime_id'].values)

In [ ]:
print(len(dType['TV']),len(dType['Movie']),len(dType['OVA']))

## Run Association Rule (By Type)

In [ ]:
!pip install efficient-apriori

In [ ]:
def get_rule_with_name(_rule,df):
    _str  = ""
    Left  = []
    Right = []
    for i in _rule.lhs:
        Left  += [" "+df[df['anime_id']==i].reset_index().loc[0,'name']]
    for j in _rule.rhs:
        Right += [" "+df[df['anime_id']==j].reset_index().loc[0,'name']]
        
    _str += '{'+", ".join(Left)+' }'
    _str += " => "+'{'+", ".join(Right)+' }'
    
    conf = "conf: {0:.3f}".format(_rule.confidence)
    supp = "supp: {0:.3f}".format(_rule.support)
    #lift = "lift: {0:.3f}".format(_rule.lift)
    #conv = "conv: {0:.3f}".format(_rule.conviction)
    #_str += "\n({}, {}, {}, {})".format(conf, supp, lift, conv)
    _str += "\n({}, {})".format(conf, supp)
    return _str

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dType['TV'],min_support=0.25, min_confidence=0.7)

In [ ]:
print(rules,len(rules))

In [ ]:
#print(rules[0].lhs, rules[0].rhs)
#print(rules)
#print(itemsets)

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dType['Movie'],min_support=0.2, min_confidence=0.5)

print(rules,len(rules))
print()

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dType['OVA'],min_support=0.05, min_confidence=0.9)

print(rules,len(rules))
print()

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()

In [ ]:
df_anime[df_anime["anime_id"] == 138]

## Run Association Rule (By genre) Comedy, Action, Adventure and Fantasy

In [ ]:
#Comedy, Action, Adventure and Fantasy
set_genre = {'Comedy':set(),'Action':set() ,'Adventure':set(),'Fantasy':set()}

for i in range(len(df_anime)):
    str_genre = df_anime.loc[i, "genre"]
    if pd.isnull(str_genre):
        print(i,end=" ")
        continue
    Arr = str_genre.strip().split(", ")
    for j in Arr:
        if j in set_genre.keys():
            set_genre[ j ].add(df_anime.loc[i,"anime_id"])


In [ ]:
dGenre = {'Comedy':[], 'Action':[], 'Adventure':[], 'Fantasy':[]}
grouped = df_rate.groupby("user_id")
k = 0
who_skipped = [46809]
#limit_rate = 100000

for i in range(73515):
    if i+1 in who_skipped:
        continue
    A = grouped.get_group(i+1)
    
    #for j in range(len(A)):
    B = A[A['rating']>=1]
    set_trans = set(B['anime_id'].values)
    
    for j in dGenre.keys():
        adding = list(set_genre[j].intersection(set_trans)) 
        if len(adding)>1:
            dGenre[j].append(adding)

In [ ]:
print("Type")
for i in dType.keys():
    print(i,len(dType[i]))
print()
print("Genre")
for i in dGenre.keys():
    print(i,len(dGenre[i]))

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dGenre['Comedy'],min_support=0.2, min_confidence=0.5)

print(rules,len(rules))
print()

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dGenre['Action'],min_support=0.25, min_confidence=0.6)

print(rules,len(rules))
print()

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dGenre['Adventure'],min_support=0.2, min_confidence=0.5)

print(rules,len(rules))
print()

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()

In [ ]:
from efficient_apriori import apriori #https://efficient-apriori.readthedocs.io/en/latest/
itemsets, rules = apriori( dGenre['Fantasy'],min_support=0.2, min_confidence=0.6)

print(rules,len(rules))
print()

for i in rules:
    print(get_rule_with_name(i,df_anime))
    print()